In [13]:
# This version is for file w/ both hcd & ethcd
# NOTE!! This version is with raw byonic excel file
# NOTE!! Here I assume byonic does not contain repeated scans
# NOTE!! has id but target not present -> 'N/A', no id at all -> -1 (align data func default) 
# file: pGlycoDB-GP-FDR-Pro-Quant-Site
# PepScore>5,GlyScore>4
# compare the results to that of byonic (score > 200, pep2d < 0.001)

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib import ticker
from IPython.display import display, HTML
import re # finding specific patterns in str
import textwrap # split text into equal parts
import collections # return repeated items in list
from collections import OrderedDict
import time
import sys

start_time = time.time()

# read in pglyco as df
pglycofile = 'pglyco_UU4_Fcwf_T_Chy_Mammalian'
pglyco_df = pd.read_excel('%s.xlsx'%pglycofile, header = 0)
pglyco_df = pglyco_df.fillna('N/A')
pglyco_df = pglyco_df.sort_values(by=['Scan'])
pglyco_df = pglyco_df.reset_index(drop = True)
# display(HTML(pglyco_df.to_html()))

## preprocess the pglycofile first
# output column name
print('original pglyco columns:\n%s\n'%pglyco_df.columns)

# record original data size
print('original pglyco data size:\nrow: %s\ncol: %s\n'%(pglyco_df.shape[0], pglyco_df.shape[1]))

# change J back to N as column named peptide(J-->N)
pglyco_df['Peptide'] = pglyco_df['Peptide'].str.replace('J','N')

# analyze sequon in pglyco file
pglyco_sequon = pglyco_df['Peptide'].str.findall('(N[ARNDBCEQZGHILKMFSTWYV]T)|(N[ARNDBCEQZGHILKMFSTWYV]S)').tolist()
pglyco_sequon_lst = []
for t in pglyco_sequon:
    t = str(t)
    res = re.findall('[ARNDBCEQZGHILKMFSTWYVP]', t)
    res = ''.join(res)
    res = textwrap.wrap(res, 3)
    if res == []:
        pglyco_sequon_lst.append('N/A')
    elif len(res) == 1:
        pglyco_sequon_lst.append(res[0])
    else:
        pglyco_sequon_lst.append(res)
        
# print('pglyco_sequon: %s'%pglyco_sequon_lst)
pglyco_df.insert(pglyco_df.columns.get_loc('Peptide') + 1 , 'Sequon', pglyco_sequon_lst , True)
# display(HTML(pglyco_df.to_html()))

# replace H, N, A, F, G symbols w/ Hex, HexNAc, NeuAc, Fuc, NeuGc
byonicstyle = []
if 'Glycan(H,N,A,G,F)' in pglyco_df.columns:
    glycan_num = pglyco_df['Glycan(H,N,A,G,F)'].tolist()
    for i in glycan_num:
        i = i.split(' ')
#         print(i)
        new_order = [1, 0, 4, 2, 3]
        i = [i[x] for x in new_order]
#         print('new: %s'%i)
        if i[0] != '0':
            n = 'HexNAc(%s)'%(i[0])
        else :
            n = ''
        if i[1] != '0':
            h = 'Hex(%s)'%(i[1])
        else:
            h = ''
        if i[2] != '0':
            f = 'Fuc(%s)'%(i[2])
        else:
            f = ''
        if i[3] != '0':
            a = 'NeuAc(%s)'%(i[3])
        else:
            a = ''
        if i[4] != '0':
            g = 'NeuGc(%s)'%(i[4])
        else:
            g = ''
        each_byonicstyle = n + h + f + a + g
        byonicstyle.append(each_byonicstyle)
elif 'Glycan(H,N,A,F)' in pglyco_df.columns:
    glycan_num = pglyco_df['Glycan(H,N,A,F)'].tolist()
    for i in glycan_num:
        i = i.split(' ')
        new_order = [1, 0, 3, 2]
        i = [i[x] for x in new_order]
        if i[0] != '0':
            n = 'HexNAc(%s)'%(i[0])
        else :
            n = ''
        if i[1] != '0':
            h = 'Hex(%s)'%(i[1])
        else:
            h = ''
        if i[2] != '0':
            f = 'Fuc(%s)'%(i[2])
        else:
            f = ''
        if i[3] != '0':
            a = 'NeuAc(%s)'%(i[3])
        else:
            a = ''
        each_byonicstyle = n + h + f + a
        byonicstyle.append(each_byonicstyle)
else:
    sys.exit('Please check if there are glycans other than H, N, A, G, F or if column name has changed. This app will stop.')
# print(byonicstyle)
pglyco_df.insert(pglyco_df.columns.get_loc('GlycanComposition') + 1 , 'GlycanComposition_ByonicStyle', byonicstyle , True)
# display(HTML(pglyco_df.to_html()))

# if the etdscan is not blank, duplicate the row and change the duplicated scan to etdscan (micmic byonic format)
pglyco_df.insert(pglyco_df.columns.get_loc('Scan') + 1 , 'FragmentType', 'hcd' , True) # insert 'fragment type' col
row_to_duplicate = pglyco_df[pglyco_df['ETDScan'] != 'N/A'].copy() # assume the missing ETDScan in pglyco is represented as blank & later filled w/ N/A
row_to_duplicate['FragmentType'] = 'ethcd'
row_to_duplicate['Scan'] = row_to_duplicate['ETDScan']
pglyco_df = pd.concat([pglyco_df, row_to_duplicate]) # duplicate w/ index
pglyco_df = pglyco_df.sort_values(by = ['Scan']) # sort by scan directly, the duplicated ethcd rows can be separated from hcd rows
# display(HTML(pglyco_df.to_html()))
print('----- pGlyco data preprocessing completed. -----\n')

## import byonic raw excel file (contain all the info.) to compare
byonicfile = 'byonic_UU4_Fcwf_T_Chy_Mammalian'
byonic_df = pd.read_excel('%s.xlsx'%byonicfile, header = 0)
byonic_df = byonic_df.fillna('N/A')
print('original byonic columns:\n%s\n'%byonic_df.columns)
# record original data size
print('original byonic data size:\nrow: %s\ncol: %s\n'%(byonic_df.shape[0], byonic_df.shape[1]))
# extract 'scan' from 'scan #' in byonic file & add a 'Scan' column
byonic_scan = byonic_df['Scan #'].tolist()
byonic_scan_lst = []
for scan in byonic_scan:
    scan = scan.split(' ')[-1].split('=')[-1]
    scan = int(scan)
#     print(scan)
    byonic_scan_lst.append(scan)
byonic_df.insert(byonic_df.columns.get_loc('Scan\r\nTime') + 1 , 'Scan', byonic_scan_lst , True)
byonic_df = byonic_df.sort_values(by = ['Scan'])
byonic_df = byonic_df.reset_index(drop = True)
# add 'PureSequence' column to byonic file
if 'Sequence\r\n(unformatted)' in byonic_df.columns: # deal w/ dif byonic version
    byonic_seq = byonic_df['Sequence\r\n(unformatted)'].str.findall('[ARNDBCEQZGHILKMFSTWYVP]').tolist()
    byonic_seq = [''.join(each_pure) for each_pure in byonic_seq]
    byonic_df.insert(byonic_df.columns.get_loc('Sequence\r\n(unformatted)') + 1 , 'PureSequence', byonic_seq , True)
elif 'Sequence' in byonic_df.columns:
    byonic_seq = byonic_df['Sequence'].str.findall('[ARNDBCEQZGHILKMFSTWYVP]').tolist()  
    byonic_seq = [''.join(each_pure) for each_pure in byonic_seq]
    byonic_df.insert(byonic_df.columns.get_loc('Sequence') + 1 , 'PureSequence', byonic_seq , True)
else:
    sys.exit('Please check if sequence column name has changed. This app will stop.')
    
# add 'Sequon' column to byonic file
byonic_sequon_lst = []
byonic_sequon = byonic_df['PureSequence'].str.findall('(N[ARNDBCEQZGHILKMFSTWYV]T)|(N[ARNDBCEQZGHILKMFSTWYV]S)').tolist()
for t in byonic_sequon:
#     print(t)
    t = str(t)
    res = re.findall('[ARNDBCEQZGHILKMFSTWYVP]', t)
    res = ''.join(res)
    res = textwrap.wrap(res, 3)
    if res == []:
        byonic_sequon_lst.append('N/A')
    elif len(res) == 1:
        byonic_sequon_lst.append(res[0])
    else:
        byonic_sequon_lst.append(res)
byonic_df.insert(byonic_df.columns.get_loc('PureSequence') + 1 , 'Sequon', byonic_sequon_lst , True)
# add N/A col 'Pair' for later modification (pair: cal.m/z same, cal.mh same, pureseq same, scan difference <= 5, hcd before ethcd)
byonic_df.insert(byonic_df.columns.get_loc('Scan\r\nTime') + 1 , 'Pair', 'N/A' , True)
# potential_pair = byonic_df[['Calc.\r\nm/z', 'Calc.\r\nMH', 'PureSequence']][byonic_df.duplicated(subset=['Calc.\r\nm/z', 'Calc.\r\nMH', 'PureSequence'], keep=False)]
potential_pair = byonic_df[byonic_df.duplicated(subset=['Calc.\r\nm/z', 'Calc.\r\nMH', 'PureSequence'], keep=False)].sort_values(['Calc.\r\nm/z'])
# print(potential_pair)
mz_gp = sorted(list(set(potential_pair['Calc.\r\nm/z'].tolist())))
pair_cnt = 0
all_hcd_ind = []
all_etd_ind = []
# mh_gp = sorted(list(set(potential_pair['Calc.\r\nMH'].tolist())))
# pureseq_gp = sorted(list(set(potential_pair['PureSequence'].tolist())))
# gp_cnt = 0
for i in mz_gp:
    gp = potential_pair[potential_pair['Calc.\r\nm/z'] == i][potential_pair.duplicated(subset=['Calc.\r\nMH', 'PureSequence'], keep=False)]
    gp = gp.sort_values(['Scan'])
#     print('gp:\n%s\n'%gp[['Scan', 'Fragment\r\nType']])
    # skip the gp w/o ethcd & find pairs: last criterion -> scan dif <= 5
    if 'hcd' in gp['Fragment\r\nType'].tolist() and 'ethcd' in gp['Fragment\r\nType'].tolist():
        # from hcd find the nearest ethcd below
        pair_candidate = gp[(gp['Fragment\r\nType'] == 'hcd') & (gp['Fragment\r\nType'].shift(-1) == 'ethcd') & (gp['Scan'].shift(-1) - gp['Scan'] <= 5)]
#         print('pair_candidate:\n%s\n'%pair_candidate[['Scan', 'Fragment\r\nType']])
        hcd_iloc = [gp.index.get_loc(ind) for ind in pair_candidate.index] # get the positions of the hcd in pairs in gp df
# #         print('hcd_ind:\n%s\n'%hcd_loc)
#         pair_lst = ['pair%s'%(i+1+pair_cnt) for i in range(len(hcd_iloc))]
# #         print('pair_lst:%s'%pair_lst)
#         gp['Pair'].iloc[hcd_iloc] = pair_lst
        etd_iloc = list(np.array(hcd_iloc) + 1)
#         gp['Pair'].iloc[etd_iloc] = pair_lst
#         pair_cnt += len(pair_lst) 
# #         print('NEW_gp:\n%s\n'%gp[['Scan', 'Fragment\r\nType', 'Pair']])
#         byonic_df.loc[gp.index.tolist()] = gp
        hcd_ind = pair_candidate.index.tolist()
#         print('hcd_ind:\n%s\n'%hcd_ind)
        etd_ind = [gp.index[i] for i in etd_iloc] 
#         print('etd_ind:\n%s\n'%etd_ind)
        all_hcd_ind.extend(hcd_ind)
        all_etd_ind.extend(etd_ind)
    else:
        pass
all_hcd_ind.sort()
all_etd_ind.sort()
# print('all_hcd_ind:\n%s\n'%all_hcd_ind)
# print('all_etd_ind:\n%s\n'%all_etd_ind)
byonic_df['Pair'].iloc[all_hcd_ind] = ['pair%s'%(i+1) for i in range(len(all_hcd_ind))]
byonic_df['Pair'].iloc[all_etd_ind] = ['pair%s'%(i+1) for i in range(len(all_etd_ind))]
# display(HTML(byonic_df.to_html()))
print('----- Byonic data preprocessing completed. -----\n')

## combine scans from the two files
print('----- Start combining pGlyco & Byonic. -----\n')
pglyco_scan = pglyco_df['Scan'].tolist()
sorted_byonic_scan = byonic_df['Scan'].tolist()
byonic_repeated_scan = [item for item, count in collections.Counter(sorted_byonic_scan).items() if count > 1]
pglyco_repeated_scan = [item for item, count in collections.Counter(pglyco_scan).items() if count > 1]
byonic_repeated_count = [count for item, count in collections.Counter(sorted_byonic_scan).items() if count > 1]
pglyco_repeated_count = [count for item, count in collections.Counter(pglyco_scan).items() if count > 1]
byonic_repeated_dict = dict(zip(byonic_repeated_scan, byonic_repeated_count))
pglyco_repeated_dict = dict(zip(pglyco_repeated_scan, pglyco_repeated_count))
print('repeated scans in byonic file {scan:times}: %s'%(byonic_repeated_dict))
print('repeated scans in pglyco file {scan:times}: %s'%(pglyco_repeated_dict))

# if the repeated scan from two files overlaps, then only choose the one w/ more times & add corresponding rows
combined_scan = byonic_scan_lst.copy()
combined_scan.extend(pglyco_scan)
combined_scan = sorted(set(combined_scan))
keys = set(byonic_repeated_dict.keys()).union(pglyco_repeated_dict.keys())
repeat = {k:max(byonic_repeated_dict.get(k,float('-inf')), pglyco_repeated_dict.get(k, float('-inf'))) for k in keys}
for k in repeat.keys():
    for i in range(repeat[k] - 1):
        combined_scan.append(k)
combined_scan = sorted(combined_scan)
# combined data based on 'Scan' in byonic & pglyco
byonic_scanasid = byonic_df.copy()
new_byonic_col = [n + '[Byonic]' if n != 'Scan' else n for n in byonic_scanasid.columns]
# print('\nnew_byonic_col:\n%s\n'%new_byonic_col)
byonic_scanasid.columns = new_byonic_col
pglyco_scanasid = pglyco_df.copy()
new_pglyco_col = [n + '[pGlyco]' if n != 'Scan' else n for n in pglyco_scanasid.columns]
pglyco_scanasid.columns = new_pglyco_col
# print('new_pglyco_col:\n%s\n'%new_pglyco_col)
byonic_scanasid = byonic_scanasid.set_index('Scan')
pglyco_scanasid = pglyco_scanasid.set_index('Scan')
# display(HTML(byonic_scanasid.to_html()))
# display(HTML(pglyco_scanasid.to_html()))
# align Index & concat
a1, a2 = byonic_scanasid.align(pglyco_scanasid, join = 'outer', axis = 0)
all_combined_df = pd.concat([a1,a2], axis = 1)
all_combined_df.reset_index(level=0, inplace=True)
# change all nan to blank -1
all_combined_df = all_combined_df.fillna(-1)
# display(HTML(all_combined_df.to_html()))

## result post-processing
# glycan comprison: only present in byonic -> b, only present in pglyco -> p, both the same -> b+p, not the same -> b/p
conditions = [
    (all_combined_df['Glycans[Byonic]'] != -1) & (all_combined_df['GlycanComposition_ByonicStyle[pGlyco]'] == -1),
    (all_combined_df['Glycans[Byonic]'] == -1) & (all_combined_df['GlycanComposition_ByonicStyle[pGlyco]'] != -1),
    (all_combined_df['Glycans[Byonic]'] != -1) & (all_combined_df['GlycanComposition_ByonicStyle[pGlyco]'] != -1) & (all_combined_df['Glycans[Byonic]'] == all_combined_df['GlycanComposition_ByonicStyle[pGlyco]']),
    (all_combined_df['Glycans[Byonic]'] != -1) & (all_combined_df['GlycanComposition_ByonicStyle[pGlyco]'] != -1) & (all_combined_df['Glycans[Byonic]'] != all_combined_df['GlycanComposition_ByonicStyle[pGlyco]'])]
choices = ['B', 'P', 'B+P', 'B/P'] 
glycan_source = np.select(conditions, choices) 
all_combined_df.insert(all_combined_df.columns.get_loc('Scan') + 1 , 'GlycanSource', glycan_source , True)

print('\ncombined data shape:\nrow --> %s, column --> %s'%(all_combined_df.shape[0], all_combined_df.shape[1]))

## style apply for excel export
# color the rows below the threshold (threshold [byonic: score > 200 & pep2d < 0.001; pglyco: PepScore>5 & GlyScore>4])

# color the background to separate byonic data from pglyco data
# color code => #ffedcc -> light orange for byonic; #add8e6 -> light blue for pglyco

def bg_color(x):
    c1 = 'background-color: #98FB98' 
    c2 = 'background-color: #add8e6'
    c3 = 'background-color: #ffedcc'
    c = '' 
    #compare columns
    b_mask = (all_combined_df['Score[Byonic]'] > 200) & (all_combined_df['PEP\r\n2D[Byonic]'].abs() < 0.001) & (all_combined_df['PepScore[pGlyco]'] <= 5) & (all_combined_df['GlyScore[pGlyco]'] <= 4)
    p_mask = (all_combined_df['Score[Byonic]'] <= 200) & (all_combined_df['PEP\r\n2D[Byonic]'].abs() >= 0.001) & (all_combined_df['PepScore[pGlyco]'] > 5) & (all_combined_df['GlyScore[pGlyco]'] > 4)    
    both_mask = (all_combined_df['Score[Byonic]'] > 200) & (all_combined_df['PEP\r\n2D[Byonic]'].abs() < 0.001) & (all_combined_df['PepScore[pGlyco]'] > 5) & (all_combined_df['GlyScore[pGlyco]'] > 4)
    #DataFrame with same index and columns names as original filled empty strings
    df1 =  pd.DataFrame(c, index=x.index, columns=x.columns)
    #modify values of df1 column by boolean mask
    df1[b_mask] = c1
    df1[p_mask] = c2
    df1[both_mask] = c3
    return df1


# all_combined_df.style.apply(bg_color, axis=None).to_excel('20210530_byonic_pglyco_combined_hcd_ethcd.xlsx', index = False)  
print("\nexecution time: --- %s seconds ---" % (time.time() - start_time))

original pglyco columns:
Index(['GlySpec', 'PepSpec', 'RawName', 'Scan', 'RT', 'PrecursorMH',
       'PrecursorMZ', 'Charge', 'Rank', 'Peptide', 'Mod', 'PeptideMH',
       'Glycan(H,N,A,G,F)', 'GlycanComposition', 'PlausibleStruct', 'GlyID',
       'GlyFrag', 'GlyMass', 'GlySite', 'TotalScore', 'PepScore', 'GlyScore',
       'CoreMatched', 'MassDeviation', 'PPM', 'GlyIonRatio', 'byIonRatio',
       'czIonRatio', 'GlyDecoy', 'PepDecoy', 'IsSmallGlycan', 'GlycanPEP',
       'GlycanFDR', 'PeptidePEP', 'PeptideFDR', 'TotalFDR', 'Proteins',
       'Genes', 'ProSites', 'MonoArea', 'IsotopeArea', 'ETDScan',
       'LocalizedSiteGroups', 'LocalizedScore', 'LocalizedIonRatio',
       'PreLocalizedScore'],
      dtype='object')

original pglyco data size:
row: 205
col: 46

----- pGlyco data preprocessing completed. -----

original byonic columns:
Index(['PID', 'Prot.\r\nRank', 'Pos.', 'Sequence', 'SequenceOriginalDebug',
       'PeptideParseFriendly', 'Calc.\r\nMH', 'Obs.\r\nm/z', 'Calc.\r\nm/z'

<ipython-input-13-abaea33c7724>:191: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  gp = potential_pair[potential_pair['Calc.\r\nm/z'] == i][potential_pair.duplicated(subset=['Calc.\r\nMH', 'PureSequence'], keep=False)]


----- Byonic data preprocessing completed. -----

----- Start combining pGlyco & Byonic. -----

repeated scans in byonic file {scan:times}: {}
repeated scans in pglyco file {scan:times}: {5852: 2, 5855: 2}

combined data shape:
row --> 8077, column --> 94
execution time: --- 177.08389377593994 seconds ---
